In [ ]:
import dash
from dash import Dash, html, dcc, dash_table
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
from dash import no_update  # Corrected import for no_update
import pandas as pd
import plotly.graph_objs as go
import io
import xlsxwriter
from lightgbm import LGBMRegressor
import numpy as np
import matplotlib.pyplot as plt
import datetime

# Initialize the Dash app
app = Dash(__name__)

# Assuming you have a list of models (e.g., different trained LightGBM models)
models = ['ARIMA_v1', 'LightGBM_v1', 'LSTM_v1']  # replace with your actual models' names or identifiers

app.layout = html.Div([
    
    html.Div([
    # Dropdown for model selection
    dcc.Dropdown(
        id='model-selector',
        options=[{'label': model, 'value': model} for model in models],
        value='LightGBM_v1',style={'width': '50%', 'justifyContent': 'center','align-items': 'center'}
    ),

    # Dropdown or input for forecasting periods
    dcc.Input(
        id='forecast-periods',
        type='number',
        value=14,style={'width': '10%', 'justifyContent': 'center','align-items': 'center'}
        
    )],style={'display': 'flex', 'width': '40%', 'justifyContent': 'center','align-items': 'center'}),

    # Button to trigger forecasting
    html.Button('Forecast', id='forecast-button',style={'width': '10%'}),

    # Graph to display the forecast
    dcc.Graph(id='forecast-graph',style={'width': '60%', 'display': 'inline-block'}),

    # Button to download data
    html.Button("Download Excel", id="btn_xlsx"),
    dcc.Download(id="download-dataframe-xlsx")
],

    style={
        'display': 'flex',
        'flex-direction': 'column',  # Stack children vertically
        'justify-content': 'space-around',  # Distribute space around items
        'align-items': 'center',  # Centers items horizontally
        'height': '100vh',  # Use full height of the view port
        'width': '80%',  # Use 80% of the width, you can adjust as needed
        'margin': '0 auto'  # Center the div on the page
    }

)

@app.callback(
    [Output('forecast-graph', 'figure'),
     Output("download-dataframe-xlsx", "data")],
    [Input('forecast-button', 'n_clicks'),
     Input("btn_xlsx", "n_clicks")],
    [State('model-selector', 'value'),
     State('forecast-periods', 'value')],
    prevent_initial_call=True)

def update_and_download(n_clicks_forecast, n_clicks_download, selected_model, forecast_periods):
    
    ctx = dash.callback_context

    if not ctx.triggered:
        raise PreventUpdate

    button_id = ctx.triggered[0]['prop_id'].split('.')[0]

        
    if button_id == 'forecast-button':
            
        df = pd.read_excel("C:\\Users\\cemal\\Desktop\\Dersler\\Data Informatics\\DI502\\Github\\Electricity_Consmption_EPIAS_data.xlsx")

        dates_dt = pd.to_datetime(df.Tarih, format='%d.%m.%Y')

        df.Tarih = dates_dt.copy()
        df = df.sort_values("Tarih")
    
        df['year']=df['Tarih'].dt.year 
        df['month']=df['Tarih'].dt.month
        df['day']=df['Tarih'].dt.day
        df['season']=df['Tarih'].dt.quarter
        df['week']=df['Tarih'].dt.week
        df['dayofweek']=df['Tarih'].dt.dayofweek
        df['hour']= df.Saat.astype("str").apply(lambda x: x[:2]).astype("int")

        df = df.sort_values(["year","month","day","hour"])

        target = "Tüketim Miktarı (MWh)"

        df.loc[:,target] =  pd.Series([item.replace(".", "").replace(",",".") for item in df.loc[:,target]]).astype("float")
        
        train = df[df.Tarih <= "2023-09-30"].reset_index(drop=True).copy()
        test = df[df.Tarih > "2023-09-30"].reset_index(drop=True).copy()
        
        x_train = train.iloc[:,-7:]
        x_test = test.iloc[:,-7:]
        y_train = train.loc[:,target]
        y_test = test.loc[:,target]
        
        eval_set = [(x_test,y_test)]
        basic_model = LGBMRegressor(random_state=42,verbose=1)
        basic_model.fit(x_train, y_train,eval_set=eval_set)

        y_predicted = pd.Series(basic_model.predict(x_test))
        y_predicted.index = range(17520,17828)

        # Create traces
        trace1 = go.Scatter(
            x = df.loc[17000:17520,"Tüketim Miktarı (MWh)"].index,
            y = df.loc[17000:17520,"Tüketim Miktarı (MWh)"].values,
            mode = 'lines',
            name = 'Actual',
            line = dict(color='blue')
        )
        
        trace2 = go.Scatter(
            x = y_predicted.index,  # Assuming predictions align with the actual index
            y = y_predicted,
            mode = 'lines',
            name = 'Prediction',
            line = dict(color='red')
        )

        # Layout
        layout = go.Layout(
            title = 'LightGBM_v1',
            xaxis = dict(title='Time'),
            yaxis = dict(title='Value'),
            legend = dict(x=0, y=1)
        )

        # Figure
        fig = go.Figure(data=[trace1, trace2], layout=layout)

        return fig, no_update
    
    elif button_id == "btn_xlsx":
        # Assuming the figure data is stored and accessible here
        # You need to ensure that the forecast data is available to be downloaded
        df = pd.DataFrame({'Forecast': [0] * forecast_periods})  # Replace with actual forecast data

        # Convert the DataFrame into an Excel file in memory
        output = io.BytesIO()
        with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
            df.to_excel(writer, sheet_name='Forecast Data')
            writer.save()

        output.seek(0)
        return no_update, dcc.send_bytes(output, filename="forecast_data.xlsx")
    else:
        raise PreventUpdate

if __name__ == '__main__':
    app.run_server(debug=False)